In [1]:
## Задание 3.3 Команды spark. Обработка данных postgres

from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
import os

#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

from datetime import datetime

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.5.1.jar") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4040") \
    .getOrCreate()


web = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/Test") \
    .option("dbtable", "web") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()


lk = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/Test") \
    .option("dbtable", "lk") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()


22/12/07 16:19:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
web.printSchema()
web.show(50)

root
 |-- id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)

+---+----------+------+-------+--------+-----+
| id| timestamp|  type|page_id|     tag| sign|
+---+----------+------+-------+--------+-----+
|  1|1637627426| visit|    101|   Sport| true|
|  1|1637621667|scroll|    101|   Sport| true|
|  1|1637621620| click|    101|   Sport| true|
|  1|1637621762| visit|    102|Politics| true|
|  1|1637628565| click|    102|Politics| true|
|  1|1637625861| visit|    103|   Sport| true|
|  2|1637628001| visit|    104|Politics| true|
|  2|1637628201| scrol|    104|Politics| true|
|  2|1637628151| click|    104|Politics| true|
|  2|1638628200| visit|    105|Business| true|
|  2|1638628226| click|    105|Business| true|
|  2|1637628317| visit|    106|Business| true|
|  2|1637628359| scrol|    106|Business| true|
|  3|1637628422| vis

In [3]:
lk.printSchema()
lk.show(50)

root
 |-- id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- fio: string (nullable = true)
 |-- dbd: date (nullable = true)
 |-- dpa: timestamp (nullable = true)

+---+-------+--------------------+----------+-------------------+
| id|user_id|                 fio|       dbd|                dpa|
+---+-------+--------------------+----------+-------------------+
|101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00|
|102|      2|Кудимов Алексей А...|1998-07-13|2021-11-22 00:00:00|
|105|      5|Музин Владимир Вл...|1995-02-10|2021-11-23 00:00:00|
+---+-------+--------------------+----------+-------------------+



In [4]:
#преобразуем timestamp в формат времени
web = web.select(*[i for i in web.columns if i != "timestamp"],
                 F.from_unixtime("timestamp").alias("event_time"))

# переименование колонок
web = web.withColumnRenamed("id","web_id")
lk = lk.withColumnRenamed("id","lk_id")

In [5]:
web.show()
lk.show()

+------+------+-------+--------+-----+-------------------+
|web_id|  type|page_id|     tag| sign|         event_time|
+------+------+-------+--------+-----+-------------------+
|     1| visit|    101|   Sport| true|2021-11-23 07:30:26|
|     1|scroll|    101|   Sport| true|2021-11-23 05:54:27|
|     1| click|    101|   Sport| true|2021-11-23 05:53:40|
|     1| visit|    102|Politics| true|2021-11-23 05:56:02|
|     1| click|    102|Politics| true|2021-11-23 07:49:25|
|     1| visit|    103|   Sport| true|2021-11-23 07:04:21|
|     2| visit|    104|Politics| true|2021-11-23 07:40:01|
|     2| scrol|    104|Politics| true|2021-11-23 07:43:21|
|     2| click|    104|Politics| true|2021-11-23 07:42:31|
|     2| visit|    105|Business| true|2021-12-04 21:30:00|
|     2| click|    105|Business| true|2021-12-04 21:30:26|
|     2| visit|    106|Business| true|2021-11-23 07:45:17|
|     2| scrol|    106|Business| true|2021-11-23 07:45:59|
|     3| visit|    101|   Sport|false|2021-11-23 07:47:0

**Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)**

In [6]:
web = web.withColumn("time_period",F.floor(F.hour("event_time") / F.lit(4)))
web.show()

+------+------+-------+--------+-----+-------------------+-----------+
|web_id|  type|page_id|     tag| sign|         event_time|time_period|
+------+------+-------+--------+-----+-------------------+-----------+
|     1| visit|    101|   Sport| true|2021-11-23 07:30:26|          1|
|     1|scroll|    101|   Sport| true|2021-11-23 05:54:27|          1|
|     1| click|    101|   Sport| true|2021-11-23 05:53:40|          1|
|     1| visit|    102|Politics| true|2021-11-23 05:56:02|          1|
|     1| click|    102|Politics| true|2021-11-23 07:49:25|          1|
|     1| visit|    103|   Sport| true|2021-11-23 07:04:21|          1|
|     2| visit|    104|Politics| true|2021-11-23 07:40:01|          1|
|     2| scrol|    104|Politics| true|2021-11-23 07:43:21|          1|
|     2| click|    104|Politics| true|2021-11-23 07:42:31|          1|
|     2| visit|    105|Business| true|2021-12-04 21:30:00|          5|
|     2| click|    105|Business| true|2021-12-04 21:30:26|          5|
|     

**Создайте витрину данных в Postgres со следующим содержанием**

1.       Id посетителя
2.       Возраст посетителя
3.       Пол посетителя (постарайтесь описать логику вычисления пола в отдельной пользовательской функции)
4.       Любимая тематика новостей
5.       Любимый временной диапазон посещений
6.       Id личного кабинета
7.       Разница в днях между созданием ЛК и датой последнего посещения. (-1 если ЛК нет)
8.       Общее кол-во посещений сайта
9.       Средняя длина сессии(сессией считаем временной промежуток, который охватывает последовательность событий, которые происходили подряд с разницей не более 5 минут).
10.   Среднее кол-во активностей в рамках одной сессии

In [7]:
from pyspark.sql.functions import udf

In [8]:
# пользовательская функция определения пола 
@udf(T.StringType())
def getGender(fio):
    if fio is None:
        return None
    surname, name, middlename = fio.split(" ")
    if ((surname[-2:] == "ов") or (surname[-2:] == "ев") or (middlename[-3:] == "вич")):
        return "m"
    else:
        return "w"

### 2. возраст

In [9]:
# определим возраст пользователя
lk = lk.withColumn("age",F.floor(F.datediff(F.current_date(),F.col("dbd"))/365.25))
lk.show()

+-----+-------+--------------------+----------+-------------------+---+
|lk_id|user_id|                 fio|       dbd|                dpa|age|
+-----+-------+--------------------+----------+-------------------+---+
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00| 22|
|  102|      2|Кудимов Алексей А...|1998-07-13|2021-11-22 00:00:00| 24|
|  105|      5|Музин Владимир Вл...|1995-02-10|2021-11-23 00:00:00| 27|
+-----+-------+--------------------+----------+-------------------+---+



In [10]:

# соединим 2 набора данных full_outer join
df_all=lk.alias("lk").join(web.alias("web"),
                          on = [F.col("lk.user_id")==F.col("web_id")],
                          how = "full_outer")

In [11]:
df_all.show(5)

+-----+-------+--------------------+----------+-------------------+---+------+------+-------+--------+----+-------------------+-----------+
|lk_id|user_id|                 fio|       dbd|                dpa|age|web_id|  type|page_id|     tag|sign|         event_time|time_period|
+-----+-------+--------------------+----------+-------------------+---+------+------+-------+--------+----+-------------------+-----------+
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00| 22|     1| visit|    101|   Sport|true|2021-11-23 07:30:26|          1|
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00| 22|     1|scroll|    101|   Sport|true|2021-11-23 05:54:27|          1|
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00| 22|     1| click|    101|   Sport|true|2021-11-23 05:53:40|          1|
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00| 22|     1| visit|    102|Politics|true|2021-11-23 05:56:02|          1|
|  101|      1|Никиф

In [12]:
# добавим к нашей таблице пол посетителя
df_all = df_all.withColumn("sex",getGender(F.col("fio")))
df_all.show(50)

+-----+-------+--------------------+----------+-------------------+----+------+------+-------+--------+-----+-------------------+-----------+----+
|lk_id|user_id|                 fio|       dbd|                dpa| age|web_id|  type|page_id|     tag| sign|         event_time|time_period| sex|
+-----+-------+--------------------+----------+-------------------+----+------+------+-------+--------+-----+-------------------+-----------+----+
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00|  22|     1| visit|    101|   Sport| true|2021-11-23 07:30:26|          1|   m|
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00|  22|     1|scroll|    101|   Sport| true|2021-11-23 05:54:27|          1|   m|
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00|  22|     1| click|    101|   Sport| true|2021-11-23 05:53:40|          1|   m|
|  101|      1|Никифоров Август ...|2000-05-05|2021-10-29 00:00:00|  22|     1| visit|    102|Politics| true|2021-11-2

In [13]:
#любимая тематика новостей
# подсчитывается самая посещаемая(visit+click) тематика по пользователю

#выберим клики и визиты
df_click = df_all.where((df_all.type== "click") | (df_all.type == "visit")) \
    .groupBy("user_id","tag").count().orderBy("user_id","tag")

# переименуем столбец
df_click = df_click.withColumnRenamed("count","count_click")
df_click.show()

# **** запись df в postgres ******
# df_click.write.format("jdbc")\
#     .option("url", "jdbc:postgresql://localhost:5432/Test") \
#     .option("driver", "org.postgresql.Driver").option("dbtable", "dfclick") \
#     .option("user", "postgres").option("password", "postgres").save()



+-------+--------+-----------+
|user_id|     tag|count_click|
+-------+--------+-----------+
|   null|Business|          1|
|   null|Politics|          3|
|   null|   Sport|          1|
|      1|Politics|          2|
|      1|   Sport|          3|
|      2|Business|          3|
|      2|Politics|          2|
|      5|Politics|          1|
|      5|   Sport|          2|
+-------+--------+-----------+



In [14]:
df_a = df_click
df_b = df_click.groupBy("user_id").agg(F.max("count_click").alias("count_click"))


In [15]:
df_a = df_a.filter(df_a["user_id"].isNotNull())
df_b = df_b.filter(df_b["user_id"].isNotNull())
df_a.createOrReplaceTempView("A")
df_b.createOrReplaceTempView("B")
df_a.show()
df_b.show()

# теперь нам нужно взять таблицу B и присоеденить к ней категорию из таблицы А

+-------+--------+-----------+
|user_id|     tag|count_click|
+-------+--------+-----------+
|      1|Politics|          2|
|      1|   Sport|          3|
|      2|Business|          3|
|      2|Politics|          2|
|      5|Politics|          1|
|      5|   Sport|          2|
+-------+--------+-----------+

+-------+-----------+
|user_id|count_click|
+-------+-----------+
|      1|          3|
|      2|          3|
|      5|          2|
+-------+-----------+



### 4. Любимая тематика новостей

In [16]:
# любимая тематика новостей
topic_like = spark.sql("SELECT B.user_id,A.tag AS topic_like "+
                              "FROM B, A " +
                              "WHERE (B.user_id = A.user_id AND B.count_click = A.count_click)")
topic_like.show()


+-------+----------+
|user_id|topic_like|
+-------+----------+
|      1|     Sport|
|      2|  Business|
|      5|     Sport|
+-------+----------+



In [17]:
# добавим в общую таблицу любимую тематику новостей
df_all = df_all.alias("A").join(topic_like.alias("B"), \
                               on = [F.col("A.user_id")==F.col("B.user_id")],\
                               how = "left").drop(F.col("B.user_id"))

### 5. Любимый временной диапазон посещений

In [19]:
#выберим какой период сколько раз был у каждого пользователя
df_A = df_all.groupBy("web_id","time_period").agg(F.count("*") \
                      .alias("count_period")).orderBy("web_id","time_period")
df_A.show()

+------+-----------+------------+
|web_id|time_period|count_period|
+------+-----------+------------+
|     1|          1|           6|
|     2|          1|           5|
|     2|          5|           2|
|     3|          1|           1|
|     3|          3|           1|
|     4|          1|           2|
|     4|          3|           1|
|     4|          5|           1|
|     5|          1|           3|
+------+-----------+------------+



In [20]:
# найдем максимальное(любимое) значение периода
df_B = df_A.groupBy("web_id").agg(F.max("count_period") \
                                  .alias("period_like")) \
                                  .orderBy("web_id")
df_B.show()

+------+-----------+
|web_id|period_like|
+------+-----------+
|     1|          6|
|     2|          5|
|     3|          1|
|     4|          2|
|     5|          3|
+------+-----------+



In [21]:
#объединенная таблица (в общую таблицу добавил любимый период)
df_all = df_all.alias("A").join(df_B.alias("B"),\
                                on = [F.col("A.web_id")==F.col("B.web_id")],\
                                how = "left").drop(F.col("B.web_id"))



### 7.   Разница в днях между созданием ЛК и датой последнего посещения. (-1 если ЛК нет)

In [23]:
#выберим дату последнего посещения пользователя
# web_id - содержит id пользователя, dpa - дата создания lk
lastvisit = df_all.groupBy("web_id","dpa").agg(F.max("event_time") \
                      .alias("last_visit")).orderBy("web_id")
#lastvisit.show()

lastvisit = lastvisit.withColumn("diff_in_days", F.datediff(F.col("last_visit"),F.col("dpa"))).fillna(value=-1)
lastvisit.show()

+------+-------------------+-------------------+------------+
|web_id|                dpa|         last_visit|diff_in_days|
+------+-------------------+-------------------+------------+
|     1|2021-10-29 00:00:00|2021-11-23 07:49:25|          25|
|     2|2021-11-22 00:00:00|2021-12-04 21:30:26|          12|
|     3|               null|2021-11-25 15:21:26|          -1|
|     4|               null|2021-12-07 21:56:39|          -1|
|     5|2021-11-23 00:00:00|2021-11-23 07:56:16|           0|
+------+-------------------+-------------------+------------+



In [24]:
#объединенная таблица (в общую таблицу добавил diff_in_days)
df_all = df_all.alias("A").join(lastvisit.alias("B"),\
                                on = [F.col("A.web_id")==F.col("B.web_id")],\
                                how = "left").drop(F.col("B.web_id"))

In [25]:
df_all.columns
web.columns

['web_id', 'type', 'page_id', 'tag', 'sign', 'event_time', 'time_period']

### 8. Общее количество посещений сайта


In [26]:
# найдем максимальное(любимое) значение периода
# df_B = df_A.groupBy("web_id").agg(F.max("count_period") \
#                                   .alias("period_like")) \
#                                   .orderBy("web_id")
total_visits = web.groupBy("web_id").agg(F.count("*").alias("total_visits"))
total_visits.show()

+------+------------+
|web_id|total_visits|
+------+------------+
|     1|           6|
|     3|           2|
|     5|           3|
|     4|           4|
|     2|           7|
+------+------------+



In [27]:
#объединенная таблица (в общую таблицу добавил total_visits)
df_all = df_all.alias("A").join(total_visits.alias("B"),\
                                on = [F.col("A.web_id")==F.col("B.web_id")],\
                                how = "left").drop(F.col("B.web_id"))

### 9. Средняя длина сессии(сессией считаем временной промежуток, который охватывает последовательность событий, которые происходили подряд с разницей не более 5 минут).

## формирование результирующего датасета

In [36]:
dashboard = df_all.select("user_id","age","sex","topic_like","period_like","lk_id","diff_in_days","total_visits").dropna()
dashboard.show()

+-------+---+---+----------+-----------+-----+------------+------------+
|user_id|age|sex|topic_like|period_like|lk_id|diff_in_days|total_visits|
+-------+---+---+----------+-----------+-----+------------+------------+
|      1| 22|  m|     Sport|          6|  101|          25|           6|
|      1| 22|  m|     Sport|          6|  101|          25|           6|
|      1| 22|  m|     Sport|          6|  101|          25|           6|
|      1| 22|  m|     Sport|          6|  101|          25|           6|
|      1| 22|  m|     Sport|          6|  101|          25|           6|
|      1| 22|  m|     Sport|          6|  101|          25|           6|
|      5| 27|  m|     Sport|          3|  105|           0|           3|
|      5| 27|  m|     Sport|          3|  105|           0|           3|
|      5| 27|  m|     Sport|          3|  105|           0|           3|
|      2| 24|  m|  Business|          5|  102|          12|           7|
|      2| 24|  m|  Business|          5|  102|     

In [37]:
# запись данных в postgres

dashboard.write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/Test") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "dashboard") \
    .mode("Overwrite")\
    .option("user", "postgres").option("password", "postgres").save()